In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "2"
import tensorflow as tf
import keras.backend.tensorflow_backend as KTF
 
config = tf.ConfigProto()  
config.gpu_options.allow_growth=True  
session = tf.Session(config=config)
 
KTF.set_session(session)

Using TensorFlow backend.


In [2]:
from preprocess import *
from utils import *
from generator import *
from hypers import *
from models import *
from evaluation import *
from fedtrain import *

In [3]:
news_index = read_news(data_root_path,'docs_train_dev.tsv')

In [4]:
train_session = read_train_clickhistory(news_index,data_root_path,'train_sam2.tsv')

In [5]:
test_session = read_test_clickhistory(news_index,data_root_path,'val_sam2.tsv')

In [6]:
train_user = parse_user(news_index,train_session)

In [7]:
test_user = parse_user(news_index,test_session)

In [8]:
train_sess, train_user_id, train_label, train_true_user_ids = get_train_input(news_index,train_session)

In [9]:
test_impressions, test_userids = get_test_input(news_index,test_session)

In [10]:
news_scoring,normalized_news_scoring = load_rank_model_news_emb(news_index,data_root_path)

In [11]:
train_user_dict = get_client_data_dict(train_true_user_ids)

In [12]:
training_data = [news_scoring,train_user_dict,train_user,train_user_id,train_sess,train_label,32]

# Federated Training

In [13]:
History = {}
Future = {}

quick_evaluation = True

lr = 1
step_ratio = 0.01
user_num = int(step_ratio*len(train_user_dict))
clipvalue = 0.5
clipvalue_infer = 1
lambd_infer = 0.1
cluster_distance = 0.5
budget = 10

lambd,sigma = convert(clipvalue,budget,user_num)

History[step_ratio] = []
Future[step_ratio] = []


lambd,sigma = convert(clipvalue,budget,user_num)

for repeat in range(10):
    
    model,user_model, user_vecs_model = create_model(lr,clipvalue)
    all_models = [user_vecs_model,model]
    Future[step_ratio].append([])
    History[step_ratio].append([])


    for step in range(100):
        FedTrain_single(user_num,lambd,cluster_distance,all_models,training_data)

        if (step+1)%25==0:
            if quick_evaluation:
                num = 10000
                f,h = FedRec_Recall(lambd_infer,clipvalue_infer,cluster_distance,user_model,user_vecs_model,test_impressions[:num],test_user,news_scoring,normalized_news_scoring)
            else:
                f,h = FedRec_Recall(lambd_infer,clipvalue_infer,cluster_distance,user_model,user_vecs_model,test_impressions,test_user,news_scoring,normalized_news_scoring)
            print(step)
            print(f,h)
            History[step_ratio][-1].append(h)
            Future[step_ratio][-1].append(f)
        print(step)
        step += 1
        













0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
{100: 0.0, 200: 0.0, 300: 0.0, 400: 0.0, 500: 0.0}
{100: 0.0, 200: 0.0, 300: 0.0, 400: 0.0, 500: 0.0}

{100: 3.610000555277917, 200: 5.9792920999817545, 300: 5.9792920999817545, 400: 6.029267112475508, 500: 6.086381412468368}
{100: 0.028961792913670292, 200: 0.0851620038271365, 300: 0.31515011790643965, 400: 0.4119147086003405, 500: 0.41795100011265074}

{100: 3.2940673561768286, 200: 5.651841008001968, 300: 5.66017225852267, 400: 5.722656637427944, 500: 5.751220924927498}
{100: 0.045762986024356835, 200: 0.0982434396918514, 300: 0.3370299200445623, 400: 0.4567938775702299, 500: 0.4818513777012439}

{100: 3.3403301229496862, 200: 5.626635579976689, 300: 5.632190209760615, 400: 5.682181877815939, 500: 5.709558267465283}
{100: 0.04691891088548941, 200: 0.1055989919595555, 300: 0.36434997540088954, 400: 0.48393555508116237, 500: 0.5232683665072193}

{100: 3.376736097021061, 200: 5.52689887429295, 300: 5.531065020191

100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
{100: 0.0, 200: 0.0, 300: 0.0, 400: 0.0, 500: 0.0}
{100: 0.0, 200: 0.0, 300: 0.0, 400: 0.0, 500: 0.0}

{100: 3.2460158809484145, 200: 5.754404543759866, 300: 5.9792920999817545, 400: 6.0792421249692605, 500: 6.404079706178656}
{100: 0.02657482018771452, 200: 0.0955735400351835, 300: 0.38936175852731897, 400: 0.392449780317854, 500: 0.3985978224828453}

{100: 2.9433217092552257, 200: 5.364412865037709, 300: 5.651841008001968, 400: 5.747650388990054, 500: 6.105894161380289}
{100: 0.043741614432816704, 200: 0.10932050529363231, 300: 0.4344600010953931, 400: 0.4463332624009504, 500: 0.45601197328514126}

{100: 2.9849853066816907, 200: 5.361275766208276, 300: 5.634967524652577, 400: 5.707177711843601, 500: 6.0196256371893755}
{100: 0.045130469330802746, 200: 0.11831619902968307, 300: 0.46084718168183025, 400: 0.4926363586317952, 500: 0.49981122017007507}

{100: 3.059181094426689, 200: 5.283084654

KeyboardInterrupt: 